In [57]:
# Include necessary imports
import os
import torch 
import pandas as pd
from torch.utils.data import DataLoader
from music21 import *
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler



In [58]:
# Preprocess the data

folder_path = 'Data/'
test = []
train = []
validation = []
for dirname in os.listdir(folder_path):
    if dirname != '.DS_Store':
        for filename in os.listdir(folder_path + dirname):
            df = pd.read_csv(folder_path + dirname + '/' + filename)
            transposed_df = df.transpose()
            if dirname == 'test':
                test.append(transposed_df)
            if dirname == 'train':
                train.append(transposed_df)
            if dirname == 'valid':
                validation.append(transposed_df)

# Model

In [59]:
class Model(torch.nn.Module):
    def __init__(self, input_size, output_size, hidden_dim=50, n_layers=2):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.lstm = torch.nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, output_size)
        
    def forward(self, x, hidden=None):
        lstm_output, (h,c) = self.lstm(x, hidden)
        model_output = self.fc(lstm_output)
        return model_output


# Train

In [60]:
def train_model(model, melody, harmonies, optimizer, criterion, num_epochs):
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        output = model(melody)
        loss = criterion(output, harmonies)
        loss.backward()
        optimizer.step()
        if (epoch + 1) % 100 == 0:
            print("Epoch: ", epoch, "Loss: ", loss)

In [61]:
criterion = torch.nn.MSELoss()
epoch = 5000
index = 0
for song in train:
    index = index + 1
    print("training song ", index)
    song = train[0]
    scaler_melody = MinMaxScaler()
    scaler_harmonies = MinMaxScaler()
    melody = torch.tensor(scaler_melody.fit_transform(song.iloc[0].values.reshape(-1,1)), dtype=torch.float32).unsqueeze(0).reshape(1,song.shape[1],1)
    harmonies = torch.tensor(scaler_harmonies.fit_transform(song.iloc[1:].values.T), dtype=torch.float32).unsqueeze(0)
    model = Model(1, harmonies.shape[2])
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    train_model(model, melody, harmonies, optimizer, criterion, epoch)

training song  1
Epoch:  99 Loss:  tensor(0.0568, grad_fn=<MseLossBackward0>)
Epoch:  199 Loss:  tensor(0.0460, grad_fn=<MseLossBackward0>)
Epoch:  299 Loss:  tensor(0.0252, grad_fn=<MseLossBackward0>)
Epoch:  399 Loss:  tensor(0.0070, grad_fn=<MseLossBackward0>)
Epoch:  499 Loss:  tensor(0.0025, grad_fn=<MseLossBackward0>)
Epoch:  599 Loss:  tensor(0.0019, grad_fn=<MseLossBackward0>)
Epoch:  699 Loss:  tensor(0.0007, grad_fn=<MseLossBackward0>)
Epoch:  799 Loss:  tensor(0.0014, grad_fn=<MseLossBackward0>)
Epoch:  899 Loss:  tensor(0.0004, grad_fn=<MseLossBackward0>)
Epoch:  999 Loss:  tensor(0.0004, grad_fn=<MseLossBackward0>)
Epoch:  1099 Loss:  tensor(0.0003, grad_fn=<MseLossBackward0>)
Epoch:  1199 Loss:  tensor(0.0002, grad_fn=<MseLossBackward0>)
Epoch:  1299 Loss:  tensor(0.0001, grad_fn=<MseLossBackward0>)
Epoch:  1399 Loss:  tensor(0.0002, grad_fn=<MseLossBackward0>)
Epoch:  1499 Loss:  tensor(0.0001, grad_fn=<MseLossBackward0>)
Epoch:  1599 Loss:  tensor(8.8175e-05, grad_fn=<M

# Hyperparameter 

In [ ]:
learning = [0.01,0.001]
n_layers= [1,2,3,4,5]
epochs= [100,1000,5000]
best_loss = float('inf')
best_params = {}


for LR in learning:
    for n_layer in n_layers:
        for epoch in epochs:
            print(f"Training with LR={LR} and n_layers={n_layer} and epochs={epoch}")
            model = Model(input_size=1, output_size=harmonies.shape[2], n_layers=n_layer)
            optimizer = torch.optim.Adam(model.parameters(), lr=LR)
            train_model(model, melody, harmonies, optimizer, criterion, epoch)
            with torch.no_grad():
                output = model(melody)
                loss = criterion(output, harmonies)
                print(f"Final Loss: {loss.item()}")        
            # Keep track of the best model (with lowest loss)
            if loss.item() < best_loss:
                best_loss = loss.item()
                best_params = {'learning': LR, 'n_layers': n_layer, 'epochs': epoch}
print("BEST: ", best_params)

In [ ]:
# tiny = pd.DataFrame([[67,62,59,43], [68,62,59,43]]).transpose()
# melody = torch.tensor(tiny.iloc[0], dtype=torch.float32).unsqueeze(0).reshape(1,2,1)
# harmonies = torch.transpose(torch.tensor(tiny.iloc[1:].values, dtype=torch.float32),0,1).unsqueeze(0)
# model = Model(1, harmonies.shape[2])
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# criterion = torch.nn.MSELoss()
# train_model(model, melody, harmonies, optimizer, criterion, 10000)

In [70]:
melody = train[0].iloc[0]
result = model(torch.tensor(melody, dtype=torch.float32).unsqueeze(0).reshape(1, train[0].shape[1], 1))
result_numpy = result.detach().numpy()
inversed = scaler_harmonies.inverse_transform(np.squeeze(result_numpy)).T
print(inversed)

score = stream.Score()
melody_part = stream.Part()
alto_part = stream.Part()
tenor_part = stream.Part()
bass_part = stream.Part()

for pitch in melody:
    melody_note = note.Note(int(pitch))
    melody_part.append(melody_note)

alto_notes = inversed[0]
tenor_notes = inversed[1]
bass_notes = inversed[2]  

for pitch in alto_notes:
    alto_note = note.Note(int(pitch.item()))
    alto_part.append(alto_note)
for pitch in tenor_notes:
     tenor_note = note.Note(int(pitch.item()))
     tenor_part.append(tenor_note)
for pitch in bass_notes:
    bass_note = note.Note(int(pitch.item()))
    bass_part.append(bass_note)

score.append(melody_part)
score.append(alto_part)
score.append(tenor_part)
score.append(bass_part)
score.show('midi')
score.write('musicxml', 'output.xml')

[[71.34917  73.88738  74.9473   75.08467  75.20721  74.8395   74.52746
  74.30749  74.209465 74.171165 74.14145  74.12845  74.111664 74.09347
  74.073105 74.05653  74.06194  74.04737  74.04065  74.034515 74.041275
  74.03543  74.03374  74.03118  74.051315 74.04441  74.04413  74.04183
  74.041595 74.04014  74.039566 74.03845  74.01663  74.02082  74.019295
  74.019356 74.00791  74.00979  74.00823  74.0079   73.9879   73.991714
  73.9899   73.98987  74.01642  74.00899  74.00841  74.00624  74.026566
  74.0203   74.02011  74.0181   74.0178   74.01648  74.01586  74.01478
  74.0362   74.03015  74.02983  74.02799  74.02759  74.026306 74.02557
  74.02446  74.00159  74.00571  74.004295 74.00416  73.98237  73.986534
  73.98489  73.98483  73.973724 73.975334 73.97384  73.9733   73.97188
  73.97095  73.96966  73.96856  73.9767   73.973175 73.97199  73.970276
  73.96907  73.967545 73.98591  73.97941  73.97797  73.975365 73.973885
  73.971695 73.96988  73.967766 73.965775 73.9636   73.93302  73.9387


WindowsPath('C:/Users/foodrunner/CS370/PolyphAI/Code/output.xml')

In [ ]:
# Finetune (hyperparameters, move around test data (refer to notes), etc)

In [ ]:
# Test with new data + evaluate

In [ ]:
# Make any other changes

In [ ]:
# Sheet music + audio (musicAI)

In [ ]:
# Create new models if time permits (follow steps 3 - 7)

In [ ]:
# Compare models

In [ ]:
# Front end ** if time permits
# - Interactive sheet music
# - musescore front end??